<a href="https://colab.research.google.com/github/hwangsae91/project_so/blob/master/exploration/221011/%5BExp_08%5D20221011.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# google colab전용
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Mecab 설치를 위한 환경구성
import os

# install konlpy, jdk, JPype
!pip install konlpy > /dev/null 2>&1
!apt-get install openjdk-8-jdk-headless -qq > /dev/null 2>&1
!pip3 install JPype1-py3 > /dev/null 2>&1

# install mecab-ko
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz > /dev/null 2>&1
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz > /dev/null 2>&1
os.chdir('/tmp/mecab-0.996-ko-0.9.2')
!./configure > /dev/null 2>&1
!make > /dev/null 2>&1
!make check > /dev/null 2>&1
!make install > /dev/null 2>&1

# install mecab-ko-dic
!apt-get install automake > /dev/null 2>&1
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz > /dev/null 2>&1
!tar -zxvf mecab-ko-dic-2.1.1-20180720.tar.gz > /dev/null 2>&1
os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')
!./autogen.sh > /dev/null 2>&1
!./configure > /dev/null 2>&1
!make > /dev/null 2>&1
!make install > /dev/null 2>&1

# install mecab-python
os.chdir('/content')
!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git > /dev/null 2>&1
os.chdir('/content/mecab-python-0.996')
!python3 setup.py build > /dev/null 2>&1
!python3 setup.py install > /dev/null 2>&1

※ 환경설정의 출력은 명령어를 통해 출력되지 않게 하였다.

# exploration 8번째 과제</br>
@ 황한용(3기/쏘카)

## 라이브러리 선언

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import Counter
from gensim.models.keyedvectors import Word2VecKeyedVectors
from konlpy.tag import Mecab
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.initializers import Constant
from typing import Dict, Tuple

## 상수선언

In [4]:
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/data/sentiment_classification" # 데이터 기본경로
DATA_PATH  = BASE_PATH + "/data" # 데이터 기본경로
MODEL_PATH = BASE_PATH + "/model/word2vec_ko.model" # 학습된 한국어 단어모델
STEP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어
BASE_DICTONARY = ['<PAD>', '<BOS>', '<UNK>','<UNUSED>'] # 단어의 기본구성
FEATURE_DATA = ["document"] # feature
MAX_NUM_WORDS = 10000 # 최대 사전 단어갯수
WORD_VECTOR_DIM = 100 # 단어 백터 차원수
pad_seq_kwargs = {
    "value":None # 추후 추가예정
    , "padding":"post"
    , "maxlen":None # 추후 추가예정
}
TRAIN_TEST_SPLIT_KWARGS = {
    "test_size":0.4, "random_state":2022
}
fit_kwargs = {
    "epochs":10 # epoch 횟수
    ,"validation_data": None # 추후 추가예정
    , "shuffle" : True # epoch당 셔플을 할지의 여부
    , "verbose":1
}
tokenizer = Mecab() # tokenize 인스턴스

## 함수

In [5]:
def load_data(
    train_data:pd.DataFrame
    , test_data:pd.DataFrame
    , step_words:list=STEP_WORDS
    , num_words:int=MAX_NUM_WORDS
) -> Tuple[
            np.ndarray
            , np.ndarray
            , np.ndarray
            , np.ndarray
            , Dict[int,str]
           , Dict[str,int]
        ]:
    """
    다음과 같은 전처리 후
    `train data(feature, target)`, `test data(feature, target)`, `단어사전(str기반, index기반)`
    을 리턴한다.

    - 데이터의 중복 제거
    - `NaN` 결측치 제거
    - 한국어 토크나이저로 토큰화
    - 불용어(`stop_words`) 제거
    - 사전`word_to_index` 구성
    - 텍스트 스트링을 사전 인덱스 스트링으로 변환

    Parameters
    ----------
    train_data : DataFrame
        학습 데이터
    train_data : DataFrame
        테스트 데이터
    step_words : list, default = `MAX_NUM_WORDS`
        불용어
    num_words : int, default = `MAX_NUM_WORDS`
        단어사전의 최대 단어 갯수

    Returns
    -------
    X_train : ndarray
        train feature data
    y_train : ndarray
        train target data
    X_test : ndarray
        test feature data
    y_test : ndarray
        test target data
    word_to_index : dict
        단어사전
    index_to_word : dict
        인덱스 기반 단어사전
    """
    # 1.
    train_data.drop_duplicates(subset=FEATURE_DATA, inplace=True)
    test_data.drop_duplicates(subset=FEATURE_DATA, inplace=True)

    # 2.
    train_data.dropna(how='any', inplace=True)
    test_data.dropna(how='any', inplace=True)

    # 3.
    X_train = [
        [word for word in tokenizer.morphs(sentence) if not word in step_words]
        for sentence in train_data['document']
    ]

    X_test = [
        [word for word in tokenizer.morphs(sentence) if not word in step_words]
        for sentence in test_data['document']
    ]
    
    # 4.
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(num_words - len(BASE_DICTONARY))
    vocab = BASE_DICTONARY + [key for key, _ in counter]

    # 5.
    word_to_index = {word:index for index, word in enumerate(vocab)}
    index_to_word = {index:word for word, index in word_to_index.items()}
        
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]
        
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))

    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index, index_to_word

####################################### 제공 함수(사용안함) #######################################

# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

## 메인

### IMDB 데이터셋

In [6]:
train_data = pd.read_table(DATA_PATH + "/ratings_train.txt")
test_data = pd.read_table(DATA_PATH + "/ratings_test.txt")
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터에 대한 설명은 아래와 같다.</br>
- id: The review id, provieded by Naver
- document: The actual review
- label: The sentiment class of the review. (0: negative, 1: positive)

In [7]:
X_train, y_train, X_test, y_test, word_to_index, index_to_word = load_data(train_data, test_data)

print(f"X_train len:{len(X_train)}")
print(f"X_test len:{len(X_test)}")

X_train len:146182
X_test len:49157


학습데이터와 테스트데이터를 로드

In [8]:
num_tokens = np.array(list(map(len,list(X_train) + list(X_test))))
# 문장길이의 평균값, 최대값, 표준편차를 계산 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

pad_seq_kwargs.update({
    "value":word_to_index["<PAD>"]
    , "maxlen":maxlen
})# padding 설정

문장길이 평균 :  15.969376315021577
문장길이 최대 :  116
문장길이 표준편차 :  12.843535456326455
pad_sequences maxlen :  41
전체 문장의 0.9342988343341575%가 maxlen 설정값 이내에 포함됩니다. 


 최대 길이를 `maxlen`으로 설정하였고 값은 (평균 + 2*표준편차)로 하여<br>
대부분의 문장이 학습이 가능하도록 설정하였다.  

In [9]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, **pad_seq_kwargs)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, **pad_seq_kwargs)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, **TRAIN_TEST_SPLIT_KWARGS)
fit_kwargs["validation_data"] = (X_val, y_val)

과적합 방지를 위해 학습과 점증을 2:8비율로 나누었다. 

### 한국어 Word2Vec 임베딩

In [10]:
word_vectors = Word2VecKeyedVectors.load(MODEL_PATH).wv

# 임의의 가중치 적용
embedding_matrix = np.random.rand(MAX_NUM_WORDS, WORD_VECTOR_DIM)

# 이미 학습된 단어가 있는 경우 학습된 백터를 넣는다. 
for i, w in list(index_to_word.items())[4:]:
    if w in word_vectors:
        embedding_matrix[i] = word_vectors[w]

[[ 4.99275217e-01  4.62225481e-01  7.48922154e-01 ...  9.06083423e-01
   6.24763435e-02  6.36164994e-01]
 [ 2.87175828e-01  3.77738968e-01  3.96647119e-01 ...  3.34706503e-03
   6.49053891e-01  8.45156999e-01]
 [ 3.28312285e-01  6.11347263e-01  1.87399977e-01 ...  9.37818915e-01
   1.87059165e-02  6.44619538e-01]
 ...
 [ 8.07251632e-02  8.92280787e-02  3.37170884e-02 ... -5.61643690e-02
   1.13671413e-03  5.08995205e-02]
 [-8.71433131e-03  6.62038382e-03 -8.34809095e-02 ... -8.07309225e-02
   1.48725137e-02  7.11738542e-02]
 [ 6.82061553e-01 -3.08079198e-02 -1.15238059e+00 ...  3.06192100e-01
   6.42204165e-01  3.65547091e-01]]


기본에 학습된 모델을 로드 뒤, 같은 단어장에 가중치를 랜덤으로 적용 후,<br>
단어장 안에 학습된 단어가 존제하면 백터의 값을 대입한다.

In [11]:
model_list = []

# max pooling만 존제하는 model 설계
model_maxpool = tf.keras.Sequential(name = "maxpool_model")
model_maxpool.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, WORD_VECTOR_DIM, input_shape=(None,)))
model_maxpool.add(tf.keras.layers.GlobalMaxPooling1D())
model_maxpool.add(tf.keras.layers.Dense(8, activation='relu'))
model_maxpool.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model_list.append(model_maxpool)

In [12]:
# CNN 1d model 설계
model_CNN = tf.keras.Sequential(name = "CNN_model")
model_CNN.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, WORD_VECTOR_DIM, input_shape=(None,)))
model_CNN.add(tf.keras.layers.Dropout(rate=0.2))
model_CNN.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model_CNN.add(tf.keras.layers.MaxPooling1D(5))
model_CNN.add(tf.keras.layers.Dropout(rate=0.2))
model_CNN.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model_CNN.add(tf.keras.layers.GlobalMaxPooling1D())
model_CNN.add(tf.keras.layers.Dense(8, activation='relu'))
model_CNN.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_list.append(model_CNN)

In [13]:
# RNN model 설계
model_RNN = tf.keras.Sequential(name = "RNN_model")
model_RNN.add(tf.keras.layers.Embedding(MAX_NUM_WORDS, WORD_VECTOR_DIM, input_shape=(None,)))
model_RNN.add(tf.keras.layers.LSTM(8))
model_RNN.add(tf.keras.layers.Dense(8, activation='relu'))
model_RNN.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_list.append(model_RNN)

In [14]:
for model in model_list:
    print(f"{model.name} start")
    model.compile(optimizer=tf.keras.optimizers.Adam(
                        learning_rate=0.00005 # running rate
                )
                ,loss='binary_crossentropy'
                ,metrics=['accuracy'])

    history_dict = model.fit(
                        X_train
                        , y_train
                        , **fit_kwargs
                    ).history

    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(acc) + 1)

    # "bo"는 "파란색 점"입니다
    plt.plot(epochs, loss, 'bo', label='Training loss')
    # b는 "파란 실선"입니다
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

    plt.clf()   # 그림을 초기화합니다

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

    # 테스트셋을 통한 모델 평가
    results = model.evaluate(X_test, y_test, verbose=2)
    print(results)

    print(f"{model.name} end\n")

maxpool_model start
Epoch 1/10
 669/2741 [======>.......................] - ETA: 6s - loss: 0.6883 - accuracy: 0.5504

KeyboardInterrupt: ignored